In [1]:
#importing libraries
import pandas as pd
import numpy as np
import random as random
import math
import warnings
warnings.filterwarnings("ignore")

In [2]:
def Dataframe_Load():
  #Supplier countries 
  # Created an empty dataframe with the required columns for suppliers
  supplier_details = pd.DataFrame(columns=['Supplier_ID','Country', 'lat', 'long', 'Volume', 'Time_taken'])
  #Reading world cities
  cities_df = pd.read_excel('worldcities.xlsx')
  df2 = cities_df.loc[(cities_df.country == "China") | (cities_df.country == "India") | (cities_df.country == "Japan")|(cities_df.country == "South Korea")|(cities_df.country == "France")]
  Supplier_Countries = df2.groupby('country').head(2)

  #Taking location o f manufracturer to find distance between other cities and US 
  us = cities_df.loc[(cities_df.country == "United States") ]
  us = us.head(1)
  return Supplier_Countries, us
  '''us["Current_Inventory_Level"] = random.randint(35000,50000)
  us["Max_Inventory_Level"] = 300000
  us["Inventory_Difference"] = us["Max_Inventory_Level"] - us['Current_Inventory_Level']
  print(us)'''

In [3]:
#Adding reqd columns and randomizing values of volume, time_taken
Supplier_Countries, us = Dataframe_Load()
def value_add(Supplier_Countries):
  #us_demand = float(us.Inventory_Difference)
  time1 = []
  vol = []
  material_volume_provided = []
  demand_satisfied = []
  count = 0
  for i in range(1,len(Supplier_Countries)+1):
    time1.append(random.randint(1,10))
    if i in [1,4,7]:
      vol.append(random.randint(1500,2000))
    elif i in [2,5,8,10]:
      vol.append(random.randint(2500,4000))
    else:
      vol.append(random.randint(750, 1000))
    #material_volume_provided.append(vol[count] - float(0.1*us_demand))
    #demand_satisfied.append(0.1*us_demand)
    #Supplier_Countries.drop(columns = "Material_Provided", inplace = True)
    #Supplier_Countries.drop(columns = "Supplier_ID", inplace = True)

    count = count + 1
    #demand_percentage.append((0.1*us['Demand']))
  time1.sort()
  Supplier_Countries["Volume"] = vol
  #Supplier_Countries["Material_Left"] = material_volume_provided
  #Supplier_Countries["Demand-Satisfied"] = demand_satisfied
  return Supplier_Countries

Supplier_Countries = value_add(Supplier_Countries)


In [4]:
from geographiclib.geodesic import Geodesic
geod = Geodesic.WGS84
def distance_calc(Supplier_Countries, us):
  #Calculated the distance of the suppliers from the manufacturing facility in USA
  distance1 = []
  lat1 = float(us["lat"])
  long1 = float(us["lng"])
  for index, row in Supplier_Countries.iterrows(): 
    lat2 = row['lat']
    long2 = row['lng']
    g = geod.Inverse(lat1, long1, lat2, long2)
    distance1.append(g['s12']/1000)
  Supplier_Countries["Distance_from_us"] = distance1
  Supplier_Countries.sort_values(by='Distance_from_us', ascending = True)
  return Supplier_Countries



'''  
#creating and visualizing network graph
import networkx as nx
from geographiclib.geodesic import Geodesic
geod = Geodesic.WGS84

geod = Geodesic.WGS84

G = nx.DiGraph()
G.add_node(0, latitude = us["lat"], longitude= us["lng"])
distance1 = []
lat1 = float(us["lat"])
long1 = float(us["lng"])
for index, row in Supplier_Countries.iterrows(): 
    lat2 = row['lat']
    long2 = row['lng']
    G.add_node(row['id'], latitude = row['lat'], longitude = row['lng'])
    g = geod.Inverse(lat1, long1, lat2, long2)
    distance1.append(g['s12']/1000)
    if row['Demand-Satisfied']<=row['Volume']:
      cost = g['s12']/1000 * row['Demand-Satisfied']
    else:
      cost = g['s12']/1000 * row['Volume']
    G.add_edge(0, row['id'], dist=g['s12']/1000, cost1 = cost)
    distance = nx.get_edge_attributes(G, 'dist')
    c = nx.get_edge_attributes(G, 'cost1')
    print(f"Distance: {distance[(0, row['id'])]}, Cost: {c}")



#print(G.nodes())  
#print(G.edges()) 
longi = nx.get_node_attributes(G, "longitude")
#print(float(longi[1])) 

Supplier_Countries["Distance_from_us"] = distance1
Supplier_Countries.sort_values(by='Distance_from_us', ascending = True)
Supplier_Countries["Time_Taken"] = time1

nx.draw(G)'''

Supplier_Countries = distance_calc(Supplier_Countries, us)

In [31]:
# Randomizing variables for each order and adding specific materials supplied by each city and theor individual material costs
def Supplier_func_Week(Supplier_dataframe):
  Var1 = 0
  vol = []
  time2 = []
  Supplier_dataframe.sort_values(by = 'country', inplace=True)
  Supplier_dataframe['Material'] = ['Material_1', 'Material_2', 'Material_3', 'Material_1', 'Material_2', 'Material_3', 'Material_1', 'Material_2', 'Material_3', 'Material_2']
  Supplier_dataframe['Material_Costs'] = [5,6,12,7.5,7,10,8,8,11,10]
  for i in range(1,len(Supplier_Countries)+1):
    time2.append(random.randint(1,10))
    if i in [1,4,7]:
      vol.append(random.randint(1500,2000))
    elif i in [2,5,8,10]:
      vol.append(random.randint(3000,3500))
    else:
      vol.append(random.randint(1000, 1250))
  Supplier_dataframe['Volume'] = vol
  Supplier_dataframe["Time_Taken"] = time2
  #Setting time from low to high depending on the distance from the manufacturing center.

  Supplier_Countries.sort_values(by='Distance_from_us', ascending = True)
  time2.sort()
  
  #supply=Supplier_dataframe.groupby("Material").Volume.sum()
  Supplier_dataframe['Total_Costs'] = (Supplier_dataframe['Material_Costs']*Supplier_dataframe["Volume"])*(Supplier_dataframe['Distance_from_us']/10000)
  #cost_var = Supplier_dataframe.groupby("Material").Total_Costs.sum()
  #citywise_cost = Supplier_dataframe.groupby(['Material','city','Distance_from_us']).Total_Costs.sum()
  Supplier_dataframe['Per_Unit_Price'] = Supplier_dataframe.Total_Costs/Supplier_dataframe.Volume
  #if len(city_name) > 0:
  #  Var1 = Supplier_dataframe.loc[Supplier_dataframe.city .isin(city_name.to_list())][['Material','city','Volume']]
    #Supplier_Countries.at((Supplier_dataframe.loc[Supplier_dataframe.city .isin(city_name.to_list())].index),'Volume') = 0
  return Supplier_dataframe
Supplier_func_Week(Supplier_Countries.copy())

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id,Volume,Distance_from_us,Material,Material_Costs,Time_Taken,Total_Costs,Per_Unit_Price
7,Shanghai,Shanghai,31.1667,121.4667,China,CN,CHN,Shanghai,admin,22118000.0,1156073548,1893,11892.421516,Material_1,5.0,1,11256.176965,5.946211
9,Guangzhou,Guangzhou,23.1288,113.2590,China,CN,CHN,Guangdong,admin,21489000.0,1156237133,3311,12899.735317,Material_2,6.0,3,25626.614180,7.739841
34,Paris,Paris,48.8566,2.3522,France,FR,FRA,Île-de-France,primary,11027000.0,1250015082,1015,5848.624736,Material_3,12.0,2,7123.624929,7.018350
937,Marseille,Marseille,43.2964,5.3700,France,FR,FRA,Provence-Alpes-Côte d’Azur,admin,868277.0,1250774071,1649,6316.399245,Material_1,7.5,4,7811.806766,4.737299
2,Delhi,Delhi,28.6667,77.2167,India,IN,IND,Delhi,admin,31870000.0,1356872604,3285,11770.780931,Material_2,7.0,7,27066.910752,8.239547
6,Mumbai,Mumbai,19.0758,72.8775,India,IN,IND,Mahārāshtra,admin,22186000.0,1356226629,1191,12554.672401,Material_3,10.0,10,14952.614830,12.554672
0,Tokyo,Tokyo,35.6839,139.7744,Japan,JP,JPN,Tōkyō,primary,39105000.0,1392685764,1685,10875.131358,Material_1,8.0,7,14659.677071,8.700105
18,Ōsaka,Osaka,34.7520,135.4582,Japan,JP,JPN,Ōsaka,admin,15490000.0,1392419823,3022,11127.301019,Material_2,8.0,8,26901.362944,8.901841
5,Seoul,Seoul,37.5600,126.9900,South Korea,KR,KOR,Seoul,primary,22394000.0,1410836482,1061,11081.715312,Material_3,11.0,7,12933.469940,12.189887
220,Busan,Busan,35.1000,129.0403,South Korea,KR,KOR,Busan,admin,3453198.0,1410601465,3166,11290.184512,Material_2,10.0,3,35744.724165,11.290185


In [6]:
Sup_df = Supplier_func_Week(Supplier_Countries)
Material_1_inventory = Sup_df.loc[Sup_df.Material == 'Material_1']
Mat1 = 9000
# Function to return the exact amount of material required based on order qty. Maximum material supplied is provided from the cheapest supplier.
# Minimum material is obtained from the most expensive supplier. 
def Material_Return(Material_inventory, M1):
  Material_inventory.sort_values(by = ['Material','Per_Unit_Price'], inplace = True)
  supplier_number = len(Material_inventory)
  mat_dict = {}
  mat_dict[Material_inventory.Material.unique()[0]] = []
  count = 0
  # Checking for material volume provided by supplier being equal to or exceeding the required volume of material
  # If it does then break the loop else keep appending the material provided into a dictionary.
  for index,row in Material_inventory.iterrows():
      count = count + 1
      if M1 - row['Volume'] == 0:
        mat_dict[row['Material']].append(row[['city','Volume','Total_Costs', 'Time_Taken']])
        break
      elif M1 - row['Volume'] < 0:
        row1 = pd.DataFrame()
        row1['city'] = [row['city']]
        row1['Volume'] = [M1]
        row1['Total_Costs'] = [M1*row['Per_Unit_Price']]
        row1['Time_Taken']=row['Time_Taken']
        mat_dict[row['Material']].append(row1.iloc[0])
        break
      else:
        M1 = M1 - row['Volume']
        mat_dict[row['Material']].append(row[['city','Volume','Total_Costs', 'Time_Taken']])
  return mat_dict

Material_Return(Material_1_inventory, Mat1)

{'Material_1': [city           Marseille
  Volume              1113
  Total_Costs      5272.61
  Time_Taken             5
  Name: 937, dtype: object,
  city           Shanghai
  Volume             1142
  Total_Costs     6790.57
  Time_Taken            9
  Name: 7, dtype: object,
  city             Tokyo
  Volume            1094
  Total_Costs    9517.91
  Time_Taken           1
  Name: 0, dtype: object]}

In [7]:
#Function to obtain dataframe slice which provided the specific material and demand for each material. 
# Returns a dictionary for each of the 3 materials provided
def percentage_distr(M1, M2, M3, df):
  Material_1_inventory = df.loc[df.Material == 'Material_1']
  Material_2_inventory = df.loc[df.Material == 'Material_2']
  Material_3_inventory = df.loc[df.Material == 'Material_3']
  return Material_Return(Material_1_inventory, M1), Material_Return(Material_2_inventory, M2), Material_Return(Material_3_inventory, M3)

#Function to create a dataframe with the required material amounts. This dataframe is regenerated every 10 days 
def Supply_run(var1):
  var2 = {}
  var3 = {}
  count = 0
  df_1 = pd.DataFrame()
  for value in var1:
    var2.update(value)
  for value in var2.keys():
    var3 = {}
    for value1 in range(0, len(var2[value])):
      var3['Material'] = value
      var3['city'] = var2[value][value1]['city']
      var3['Volume'] = var2[value][value1]['Volume']
      var3['Total_Costs'] = var2[value][value1]['Total_Costs']
      var3['Time_Taken'] = var2[value][value1]['Time_Taken']
      df_1 = df_1.append([var3], ignore_index= True)
  return df_1

In [8]:
#Manufacturing inventory 
def manufacturing_inventory_load():
  in_fac = []

  max_storage = {"Material1":10000, 'Material2':20000, 'Material3':10000}
  manu_capacity_per_day = 40
  unit = {"Material1": 10, "Material2":20, "Material3":5}
  needed_daily = {"Material1":400, 'Material2':800, 'Material3':200}
  needed_weekly = {"Material1":2800, 'Material2':5600, 'Material3':1400}

  for i in range(0,3):
    in_fac.append(random.randint(5000,7000))


  m_inventory = {'Name':['Material1', 'Material2', 'Material3'],
          'In_Factory': in_fac, 'Needed_Daily': list(needed_daily.values()), "Needed_Weekly" : list(needed_weekly.values())}

  deficit = [0,0,0]

  #print(m_inventory['In_Factory'])
  # Create DataFrame
  manufacturing_inventory = pd.DataFrame(m_inventory)
  #print(manufacturing_inventory.loc[manufacturing_inventory['Name']=='Material1'].Needed_Weekly)
  
  # Print the output.   
  return manufacturing_inventory
manufacturing_inventory_load()

,Name,In_Factory,Needed_Daily,Needed_Weekly
0,Material1,6120,400,2800
1,Material2,6218,800,5600
2,Material3,5405,200,1400


In [9]:
#Days to complete the order 
def days_to_make_items(qty):
  manu_capacity_per_day = 40
  days_needed = math.ceil(qty/manu_capacity_per_day)
  return days_needed

#Number of materials needed to complete the order 
def qty_to_make_items(manufacturing_inventory, quantity):
  need1 = 0
  need2 = 0
  need3 = 0
  unit = {"Material1": 10, "Material2":20, "Material3":5}
  m1_needed = unit['Material1']*quantity
  m2_needed = unit['Material2']*quantity
  m3_needed = unit['Material3']*quantity
  if manufacturing_inventory['In_Factory'][0] < m1_needed:
    need1 = m1_needed - manufacturing_inventory['In_Factory'][0]
    print(f"Material 1 needed: {need1}")
  else:
    print(f"Material 1 present: {m1_needed}")
  if manufacturing_inventory['In_Factory'][1] < m2_needed:
    need2 = m2_needed - manufacturing_inventory['In_Factory'][1]
    print(f"Material 2 needed: {need2}")
  else:
    print(f"Material 2 present: {m2_needed}")
  if manufacturing_inventory['In_Factory'][2] < m3_needed:
    need3 = m3_needed - manufacturing_inventory['In_Factory'][2]
    print(f"Material 3 needed: {need3}")
  else:
    print(f"Material 3 present: {m3_needed}")
  return need1, need2, need3

In [10]:
#Seeing for random order numbers

def orders(manufacturing_inven, qty):
  mat_req = []
  #for i in range(0,1):
  req = random.randint(400,1000)
  print(f"For order qty: {qty}")
  M1,M2,M3 = qty_to_make_items(manufacturing_inven, qty)
  mat_req.append(M1)
  mat_req.append(M2)
  mat_req.append(M3)
  return(mat_req)
  
#orders
#mat_req

In [11]:
#daily simulation

#days = math.floor(min(manufacturing_inventory['In_Factory']/manufacturing_inventory['Needed Daily']))
#days


In [12]:
#to_supplier = [orders()]

In [13]:
# days to complete the order

def days_to_complete(order_qty):
  manu_capacity_per_day = 40
  days_to_complete = math.ceil(order_qty/manu_capacity_per_day)
  return days_to_complete

In [14]:
#Reducing inventory level each day
'''
day = 1
flag = 0
count = 1
Df1 = Supplier_func_Week(Supplier_Countries.copy())
for i in range(0,days_to_complete(600)):
  print(f"Day : {day}")
  for index, row in manufacturing_inventory.iterrows():
    count +=1
    sub = row['In_Factory']- row['Needed Daily']
    if sub<= 0 :
      manufacturing_inventory.at[index,'In_Factory'] = 0
      flag = 1
    else:
      #manufacturing_inventory.iloc[index].replace(to_replace=row['In_Factory'], value = sub)
      manufacturing_inventory.at[index,'In_Factory'] = sub
  count = count + 1
  day+=1
  print(manufacturing_inventory)
  if flag ==1:
    break

'''
#deficit

'\nday = 1\nflag = 0\ncount = 1\nDf1 = Supplier_func_Week(Supplier_Countries.copy())\nfor i in range(0,days_to_complete(600)):\n  print(f"Day : {day}")\n  for index, row in manufacturing_inventory.iterrows():\n    count +=1\n    sub = row[\'In_Factory\']- row[\'Needed Daily\']\n    if sub<= 0 :\n      manufacturing_inventory.at[index,\'In_Factory\'] = 0\n      flag = 1\n    else:\n      #manufacturing_inventory.iloc[index].replace(to_replace=row[\'In_Factory\'], value = sub)\n      manufacturing_inventory.at[index,\'In_Factory\'] = sub\n  count = count + 1\n  day+=1\n  print(manufacturing_inventory)\n  if flag ==1:\n    break\n\n'

In [15]:
#Reducing inventory level each day
'''
day = 1
flag = 0
count = 1
Df1 = Supplier_func_Week(Supplier_Countries.copy())
Df2 = Supply_run(percentage_distr(mat_req[0],mat_req[1],mat_req[2],Df1))
for i in range(0,days_to_complete(900)):
  supply_df = Df2.loc[Df2.Time_Taken == i]
  if len(supply_df) > 0:
    if len(supply_df.loc[supply_df.Material == 'Material_1']) > 0:
      manufacturing_inventory.at[0, 'In_Factory'] = manufacturing_inventory.iloc[0].In_Factory + supply_df.loc[supply_df.Material == 'Material_1'].Volume.sum()
    if len(supply_df.loc[supply_df.Material == 'Material_2']) > 0:
      manufacturing_inventory.at[1, 'In_Factory'] = manufacturing_inventory.iloc[1].In_Factory + supply_df.loc[supply_df.Material == 'Material_2'].Volume.sum()
      if len(supply_df.loc[supply_df.Material == 'Material_3']) > 0:
        manufacturing_inventory.at[2, 'In_Factory'] = manufacturing_inventory.iloc[2].In_Factory + supply_df.loc[supply_df.Material == 'Material_3'].Volume.sum()
  print(supply_df)
  print(f"Day : {day}")
  for index, row in manufacturing_inventory.iterrows():
    count +=1
    sub = row['In_Factory']- row['Needed Daily']
    if sub<= 0 :
      manufacturing_inventory.at[index,'In_Factory'] = 0
      flag = 1
    else:
      #manufacturing_inventory.iloc[index].replace(to_replace=row['In_Factory'], value = sub)
      manufacturing_inventory.at[index,'In_Factory'] = sub
  count = count + 1
  day+=1
  print(manufacturing_inventory)
  if flag ==1:
    break
'''

'\nday = 1\nflag = 0\ncount = 1\nDf1 = Supplier_func_Week(Supplier_Countries.copy())\nDf2 = Supply_run(percentage_distr(mat_req[0],mat_req[1],mat_req[2],Df1))\nfor i in range(0,days_to_complete(900)):\n  supply_df = Df2.loc[Df2.Time_Taken == i]\n  if len(supply_df) > 0:\n    if len(supply_df.loc[supply_df.Material == \'Material_1\']) > 0:\n      manufacturing_inventory.at[0, \'In_Factory\'] = manufacturing_inventory.iloc[0].In_Factory + supply_df.loc[supply_df.Material == \'Material_1\'].Volume.sum()\n    if len(supply_df.loc[supply_df.Material == \'Material_2\']) > 0:\n      manufacturing_inventory.at[1, \'In_Factory\'] = manufacturing_inventory.iloc[1].In_Factory + supply_df.loc[supply_df.Material == \'Material_2\'].Volume.sum()\n      if len(supply_df.loc[supply_df.Material == \'Material_3\']) > 0:\n        manufacturing_inventory.at[2, \'In_Factory\'] = manufacturing_inventory.iloc[2].In_Factory + supply_df.loc[supply_df.Material == \'Material_3\'].Volume.sum()\n  print(supply_df)\

In [16]:
'''
#Reducing inventory level each day
if __name__ == "__main__":
  Supplier_Countries, us = Dataframe_Load()
  Supplier_Countries = value_add(Supplier_Countries)
  Supplier_Countries = distance_calc(Supplier_Countries, us)
  manufacturing_inventory = manufacturing_inventory_load()
  day = 1
  flag = 0
  count_days = -1
  inventory = 0
  losses = 0
  costs = 0
  count = 0
  qty = random.randint(4000,10000)
  days_lost = 0
  Df1 = Supplier_func_Week(Supplier_Countries.copy())
  list_req2 = []
  list_req2 = orders(manufacturing_inventory, qty)
  print(Df1)
  Df2 = Supply_run(percentage_distr(list_req2[0],list_req2[1],list_req2[2],Df1))
  print(Df2)
  

  for i in range(0,1000):
    for i in range(0,days_to_complete(qty)):
      count_days = count_days + 1
      supply_df = Df2.loc[Df2.Time_Taken == i]
      if count_days%10 ==0 and count_days > 0:
        list_req = orders(manufacturing_inventory, (qty-inventory))
        Df2 = Supply_run(percentage_distr(list_req[0],list_req[1],list_req[2],Df1))
        no_of_iter = count_days/10
        Df2['Time_Taken'] = Df2.Time_Taken + (10*no_of_iter) 
        print(Df2) 
      print("The materials supplied this iteration are:- \n", supply_df)
      if len(supply_df) > 0:
        if len(supply_df.loc[supply_df.Material == 'Material_1']) > 0:
          manufacturing_inventory.at[0, 'In_Factory'] = manufacturing_inventory.iloc[0].In_Factory + supply_df.loc[supply_df.Material == 'Material_1'].Volume.sum()
          costs = costs +  supply_df.loc[supply_df.Material == 'Material_1'].Total_Costs.sum()   
        if len(supply_df.loc[supply_df.Material == 'Material_2']) > 0:
          manufacturing_inventory.at[1, 'In_Factory'] = manufacturing_inventory.iloc[1].In_Factory + supply_df.loc[supply_df.Material == 'Material_2'].Volume.sum()
          costs = costs +  supply_df.loc[supply_df.Material == 'Material_2'].Total_Costs.sum()
        if len(supply_df.loc[supply_df.Material == 'Material_3']) > 0:
          manufacturing_inventory.at[2, 'In_Factory'] = manufacturing_inventory.iloc[2].In_Factory + supply_df.loc[supply_df.Material == 'Material_3'].Volume.sum()
          costs = costs +  supply_df.loc[supply_df.Material == 'Material_3'].Total_Costs.sum()
      print(costs)
      print(f"Day : {day}")

      for index, row in manufacturing_inventory.iterrows():
        if min(manufacturing_inventory["In_Factory"]) <= 0:
          flag = 1
          break  
        count +=1
        sub = row['In_Factory']- row['Needed Daily']
        if sub<= 0 :
          manufacturing_inventory.at[index,'In_Factory'] = 0
          flag = 1
        else:
          #manufacturing_inventory.iloc[index].replace(to_replace=row['In_Factory'], value = sub)
          manufacturing_inventory.at[index,'In_Factory'] = sub
          flag = 0
      day+=1
      print(manufacturing_inventory)
      if flag !=1:
        inventory = inventory + 40
      else:
        days_lost = days_lost + 1
        losses = losses + 40
        print("Inventory is: ", inventory)
    
  
  #---------------------#

  
  try:
    
    print("Number of days taken: ", day)
    print("Number of goods made are: ", inventory)
    print("Number of days goods could not be made: ", days_lost )
    print("Total Cost of the order is: ", costs)
    print("Price per good is: ", costs/inventory)
    var0 = costs/qty
    print("Losses made are: ", losses*var0)
    print(f"Optimized cost is: {(costs/3)}")
    print(f"Optimized days lost is: {(days_lost/3)}")
    print(f"Optimized price per good is: {(costs/inventory/3)}")
    print(f"Optimized loss per good is: {(losses*var0/3)}")
  except:
    pass
'''


'\n#Reducing inventory level each day\nif __name__ == "__main__":\n  Supplier_Countries, us = Dataframe_Load()\n  Supplier_Countries = value_add(Supplier_Countries)\n  Supplier_Countries = distance_calc(Supplier_Countries, us)\n  manufacturing_inventory = manufacturing_inventory_load()\n  day = 1\n  flag = 0\n  count_days = -1\n  inventory = 0\n  losses = 0\n  costs = 0\n  count = 0\n  qty = random.randint(4000,10000)\n  days_lost = 0\n  Df1 = Supplier_func_Week(Supplier_Countries.copy())\n  list_req2 = []\n  list_req2 = orders(manufacturing_inventory, qty)\n  print(Df1)\n  Df2 = Supply_run(percentage_distr(list_req2[0],list_req2[1],list_req2[2],Df1))\n  print(Df2)\n  \n\n  for i in range(0,1000):\n    for i in range(0,days_to_complete(qty)):\n      count_days = count_days + 1\n      supply_df = Df2.loc[Df2.Time_Taken == i]\n      if count_days%10 ==0 and count_days > 0:\n        list_req = orders(manufacturing_inventory, (qty-inventory))\n        Df2 = Supply_run(percentage_distr(list

In [33]:

#Reducing inventory level each day
if __name__ == "__main__":
  try:
      no_of_iterations = int(input("Please enter the number of times you wish for the simulation to run: "))
  except:
    print("Please enter an integer")
  Supplier_Countries, us = Dataframe_Load()
  Supplier_Countries = value_add(Supplier_Countries)
  Supplier_Countries = distance_calc(Supplier_Countries, us)
  manufacturing_inventory = manufacturing_inventory_load()
  day = 1
  flag = 0
  count = 0
  count_days = -1
  inventory = 0
  losses = 0
  costs = 0
  no_ordered = 0
  qty = random.randint(4000,10000)
  days_lost = 0
  Df1 = Supplier_func_Week(Supplier_Countries.copy())
  list_req2 = []
  list_req2 = orders(manufacturing_inventory, qty)
  print(list_req2)
  Df2 = Supply_run(percentage_distr(list_req2[0],list_req2[1],list_req2[2],Df1))
  print(Df2)

  for j in range(0,no_of_iterations):
    qty = random.randint(4000,10000)
    no_ordered += qty
    for i in range(0,days_to_complete(qty)):
      count_days = count_days + 1
      supply_df = Df1.loc[Df1.Time_Taken == i]
      if count_days%10 ==0 and count_days > 0:
        list_req = orders(manufacturing_inventory, (qty-inventory))
        Df1 = Supplier_func_Week(Supplier_Countries.copy())
        print('material3 required: ', list_req[2])
        no_of_iter = count_days/10
        Df1['Time_Taken'] = Df1.Time_Taken + (10*no_of_iter) 
        print(Df1) 
      print("The materials supplied this iteration are:- \n", supply_df)
      if len(supply_df) > 0:
        if len(supply_df.loc[supply_df.Material == 'Material_1']) > 0:
          manufacturing_inventory.at[0, 'In_Factory'] = manufacturing_inventory.iloc[0].In_Factory + supply_df.loc[supply_df.Material == 'Material_1'].Volume.sum()
          costs = costs +  supply_df.loc[supply_df.Material == 'Material_1'].Total_Costs.sum()   
        if len(supply_df.loc[supply_df.Material == 'Material_2']) > 0:
          manufacturing_inventory.at[1, 'In_Factory'] = manufacturing_inventory.iloc[1].In_Factory + supply_df.loc[supply_df.Material == 'Material_2'].Volume.sum()
          costs = costs +  supply_df.loc[supply_df.Material == 'Material_2'].Total_Costs.sum()
        if len(supply_df.loc[supply_df.Material == 'Material_3']) > 0:
          manufacturing_inventory.at[2, 'In_Factory'] = manufacturing_inventory.iloc[2].In_Factory + supply_df.loc[supply_df.Material == 'Material_3'].Volume.sum()
          costs = costs +  supply_df.loc[supply_df.Material == 'Material_3'].Total_Costs.sum()
      print(costs)
      print(f"Day : {day}")

      for index, row in manufacturing_inventory.iterrows():
        if min(manufacturing_inventory["In_Factory"]) <= 0:
          flag = 1
          break  
        count +=1
        sub = row['In_Factory']- row['Needed_Daily']
        if sub<= 0 :
          manufacturing_inventory.at[index,'In_Factory'] = 0
          flag = 1
        else:
          #manufacturing_inventory.iloc[index].replace(to_replace=row['In_Factory'], value = sub)
          manufacturing_inventory.at[index,'In_Factory'] = sub
          flag = 0
      day+=1
      print(manufacturing_inventory)
      if flag !=1:
        inventory = inventory + 40
      else:
        days_lost = days_lost + 1
        losses = losses + 40
        print("Inventory is: ", inventory)



  #print("Number of goods made are: ", inventory)
  #print("Number of days goods could not be made: ", days_lost )
  #print("Total Cost of the order is: ", costs)

  list_days_lost = []
  list_days_lost.append(days_lost)

  total_sales_value = costs/no_ordered
  Total_losses_made = losses*total_sales_value
  #print("Price per good to counter the loss is: ", costs/inventory)
  #print("Price the godds should have been if order quantity could be completed: ", costs/qty)
  var0 = costs/inventory
  print("Total Losses made are: ", Total_losses_made)
  print(f"Average Non Optimized cost for order quantity of {no_ordered} is: {(costs/no_of_iterations)}")
  print(f"Average Non Optimized days lost is: {(days_lost/no_of_iterations)}")
  print(f"Average Non Optimized price per good is: {(var0)}")
  print("Average Price the goods should have been is: ", total_sales_value)


Please enter the number of times you wish for the simulation to run: 2
For order qty: 9659
Material 1 needed: 90731
Material 2 needed: 188097
Material 3 needed: 42230
[90731, 188097, 42230]
     Material       city  Volume   Total_Costs  Time_Taken
0  Material_1  Marseille    1779   8427.655692           8
1  Material_1   Shanghai    1573   9353.389523           1
2  Material_1      Tokyo    1595  13876.667613           8
3  Material_2  Guangzhou    3480  26934.647341           7
4  Material_2      Delhi    3038  25031.742729           5
5  Material_2      Ōsaka    3360  29910.185140           2
6  Material_2      Busan    3233  36501.166528          10
7  Material_3      Paris    1179   8274.634277           2
8  Material_3      Seoul    1194  14554.724891           5
9  Material_3     Mumbai    1127  14149.115796           5
The materials supplied this iteration are:- 
 Empty DataFrame
Columns: [city, city_ascii, lat, lng, country, iso2, iso3, admin_name, capital, population, id, Vol

        Name  In_Factory  Needed_Daily  Needed_Weekly
0  Material1        8454           400           2800
1  Material2       14861           800           5600
2  Material3        7673           200           1400
For order qty: 8611
Material 1 needed: 77656
Material 2 needed: 157359
Material 3 needed: 35382
material3 required:  35382
          city city_ascii      lat       lng      country iso2 iso3  \
7     Shanghai   Shanghai  31.1667  121.4667        China   CN  CHN   
9    Guangzhou  Guangzhou  23.1288  113.2590        China   CN  CHN   
34       Paris      Paris  48.8566    2.3522       France   FR  FRA   
937  Marseille  Marseille  43.2964    5.3700       France   FR  FRA   
2        Delhi      Delhi  28.6667   77.2167        India   IN  IND   
6       Mumbai     Mumbai  19.0758   72.8775        India   IN  IND   
0        Tokyo      Tokyo  35.6839  139.7744        Japan   JP  JPN   
18       Ōsaka      Osaka  34.7520  135.4582        Japan   JP  JPN   
5        Seoul      Se

        Name  In_Factory  Needed_Daily  Needed_Weekly
0  Material1       10686           400           2800
1  Material2       24091           800           5600
2  Material3       11525           200           1400
The materials supplied this iteration are:- 
 Empty DataFrame
Columns: [city, city_ascii, lat, lng, country, iso2, iso3, admin_name, capital, population, id, Volume, Distance_from_us, Material, Material_Costs, Time_Taken, Total_Costs, Per_Unit_Price]
Index: []
747580.3152884708
Day : 42
        Name  In_Factory  Needed_Daily  Needed_Weekly
0  Material1       10286           400           2800
1  Material2       23291           800           5600
2  Material3       11325           200           1400
The materials supplied this iteration are:- 
          city city_ascii      lat       lng country iso2 iso3 admin_name  \
9   Guangzhou  Guangzhou  23.1288  113.2590   China   CN  CHN  Guangdong   
18      Ōsaka      Osaka  34.7520  135.4582   Japan   JP  JPN      Ōsaka   

   ca

The materials supplied this iteration are:- 
     city city_ascii    lat     lng      country iso2 iso3 admin_name  capital  \
5  Seoul      Seoul  37.56  126.99  South Korea   KR  KOR      Seoul  primary   

   population          id  Volume  Distance_from_us    Material  \
5  22394000.0  1410836482    1011      11081.715312  Material_3   

   Material_Costs  Time_Taken   Total_Costs  Per_Unit_Price  
5            11.0        62.0  12323.975598       12.189887  
1171891.2242862207
Day : 63
        Name  In_Factory  Needed_Daily  Needed_Weekly
0  Material1       12445           400           2800
1  Material2       35688           800           5600
2  Material3       14740           200           1400
The materials supplied this iteration are:- 
     city city_ascii      lat      lng country iso2 iso3 admin_name capital  \
2  Delhi      Delhi  28.6667  77.2167   India   IN  IND      Delhi   admin   

   population          id  Volume  Distance_from_us    Material  \
2  31870000.0  135

           city city_ascii      lat   lng country iso2 iso3  \
937  Marseille  Marseille  43.2964  5.37  France   FR  FRA   

                     admin_name capital  population          id  Volume  \
937  Provence-Alpes-Côte d’Azur   admin    868277.0  1250774071    1566   

     Distance_from_us    Material  Material_Costs  Time_Taken  Total_Costs  \
937       6316.399245  Material_1             7.5        89.0  7418.610913   

     Per_Unit_Price  
937        4.737299  
1679728.8853571827
Day : 90
        Name  In_Factory  Needed_Daily  Needed_Weekly
0  Material1       17090           400           2800
1  Material2       49601           800           5600
2  Material3       18316           200           1400
For order qty: 5811
Material 1 needed: 41020
Material 2 needed: 66619
Material 3 needed: 10739
material3 required:  10739
          city city_ascii      lat       lng      country iso2 iso3  \
7     Shanghai   Shanghai  31.1667  121.4667        China   CN  CHN   
9    Guangzhou

...

The materials supplied this iteration are:- 
      city city_ascii     lat       lng country iso2 iso3 admin_name capital  \
18  Ōsaka      Osaka  34.752  135.4582   Japan   JP  JPN      Ōsaka   admin   

    population          id  Volume  Distance_from_us    Material  \
18  15490000.0  1392419823    3368      11127.301019  Material_2   

    Material_Costs  Time_Taken   Total_Costs  Per_Unit_Price  
18             8.0       201.0  29981.399866        8.901841  
3767897.3830031357
Day : 202
        Name  In_Factory  Needed_Daily  Needed_Weekly
0  Material1       31058           400           2800
1  Material2      105649           800           5600
2  Material3       32708           200           1400
The materials supplied this iteration are:- 
 Empty DataFrame
Columns: [city, city_ascii, lat, lng, country, iso2, iso3, admin_name, capital, population, id, Volume, Distance_from_us, Material, Material_Costs, Time_Taken, Total_Costs, Per_Unit_Price]
Index: []
3767897.3830031357
Day : 2

4126339.095120577
Day : 223
        Name  In_Factory  Needed_Daily  Needed_Weekly
0  Material1       35211           400           2800
1  Material2      110995           800           5600
2  Material3       36302           200           1400
The materials supplied this iteration are:- 
 Empty DataFrame
Columns: [city, city_ascii, lat, lng, country, iso2, iso3, admin_name, capital, population, id, Volume, Distance_from_us, Material, Material_Costs, Time_Taken, Total_Costs, Per_Unit_Price]
Index: []
4126339.095120577
Day : 224
        Name  In_Factory  Needed_Daily  Needed_Weekly
0  Material1       34811           400           2800
1  Material2      110195           800           5600
2  Material3       36102           200           1400
The materials supplied this iteration are:- 
     city city_ascii    lat     lng      country iso2 iso3 admin_name  capital  \
5  Seoul      Seoul  37.56  126.99  South Korea   KR  KOR      Seoul  primary   

   population          id  Volume  Distanc

The materials supplied this iteration are:- 
 Empty DataFrame
Columns: [city, city_ascii, lat, lng, country, iso2, iso3, admin_name, capital, population, id, Volume, Distance_from_us, Material, Material_Costs, Time_Taken, Total_Costs, Per_Unit_Price]
Index: []
4406199.521715868
Day : 252
        Name  In_Factory  Needed_Daily  Needed_Weekly
0  Material1       28929           400           2800
1  Material2      110471           800           5600
2  Material3       33998           200           1400
The materials supplied this iteration are:- 
 Empty DataFrame
Columns: [city, city_ascii, lat, lng, country, iso2, iso3, admin_name, capital, population, id, Volume, Distance_from_us, Material, Material_Costs, Time_Taken, Total_Costs, Per_Unit_Price]
Index: []
4406199.521715868
Day : 253
        Name  In_Factory  Needed_Daily  Needed_Weekly
0  Material1       28529           400           2800
1  Material2      109671           800           5600
2  Material3       33798           200      

        Name  In_Factory  Needed_Daily  Needed_Weekly
0  Material1       14129           400           2800
1  Material2       80871           800           5600
2  Material3       26598           200           1400
The materials supplied this iteration are:- 
 Empty DataFrame
Columns: [city, city_ascii, lat, lng, country, iso2, iso3, admin_name, capital, population, id, Volume, Distance_from_us, Material, Material_Costs, Time_Taken, Total_Costs, Per_Unit_Price]
Index: []
4406199.521715868
Day : 290
        Name  In_Factory  Needed_Daily  Needed_Weekly
0  Material1       13729           400           2800
1  Material2       80071           800           5600
2  Material3       26398           200           1400
For order qty: -7553
Material 1 present: -75530
Material 2 present: -151060
Material 3 present: -37765
material3 required:  0
          city city_ascii      lat       lng      country iso2 iso3  \
7     Shanghai   Shanghai  31.1667  121.4667        China   CN  CHN   
9    Guangz

        Name  In_Factory  Needed_Daily  Needed_Weekly
0  Material1           0           400           2800
1  Material2       52871           800           5600
2  Material3       19598           200           1400
Inventory is:  12960
The materials supplied this iteration are:- 
 Empty DataFrame
Columns: [city, city_ascii, lat, lng, country, iso2, iso3, admin_name, capital, population, id, Volume, Distance_from_us, Material, Material_Costs, Time_Taken, Total_Costs, Per_Unit_Price]
Index: []
4406199.521715868
Day : 334
        Name  In_Factory  Needed_Daily  Needed_Weekly
0  Material1           0           400           2800
1  Material2       52871           800           5600
2  Material3       19598           200           1400
Inventory is:  12960
The materials supplied this iteration are:- 
 Empty DataFrame
Columns: [city, city_ascii, lat, lng, country, iso2, iso3, admin_name, capital, population, id, Volume, Distance_from_us, Material, Material_Costs, Time_Taken, Total_Costs, Pe